__import__ required Python libraries

In [1]:
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import requests

In [2]:
# list_teams = ['ANA', 'ARI', 'BOS', 'BUF', 'CAR', 'CBJ', 'CGY', 'CHI', 'COL', 'DAL', 'DET', 'EDM', 'FLA', 'LAK',
#              'MIN', 'MTL', 'NJD', 'NSH', 'NYI', 'NYR', 'OTT', 'PHI', 'PIT', 'SJS', 'STL', 'TBL', 'TOR', 'VAN',
#              'VEG', 'WPG', 'WSH']

__generate__ the hockey-reference url

In [3]:
team_id = 'ANA'
team_year = 2018
stats_webpage = 'https://www.hockey-reference.com/teams/{}/{}.html'.format(team_id, team_year)

__create__ the soup object

In [4]:
r = requests.get(stats_webpage)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

__get__ all records from the goalies table

In [5]:
roster_table = soup.find(id='roster').find_all('td')
roster_tbody = [cell.get_text() for cell in roster_table]
roster_records = [tuple(roster_tbody[row: row + 12]) for row in range(0, len(roster_tbody), 12)]

In [6]:
player_number_table = soup.find(id='roster').find_all('th', {'data-stat': 'number'})
player_number_list = [cell.get_text() for cell in player_number_table]
del player_number_list[0]

In [7]:
player_id_list = []

for player_id in range(len(roster_table)):
    try:
        player_id_list.append(roster_table[player_id]['data-append-csv'])
    except:
        pass

In [8]:
columns_list = ['Player',  'Flag', 'Position', 'Age', 'Height', 'Weight', 'Shoots/Catches',
                'Years Experience', 'Birth Date', 'Summary', 'Salary', 'Draft']
df_roster = pd.DataFrame.from_records(roster_records, columns=columns_list)

In [9]:
df_player_number = pd.DataFrame(player_number_list, columns=['Number'])
df_player_id = pd.DataFrame(player_id_list, columns=['Player ID'])
df_concat = pd.concat([df_player_id, df_player_number, df_roster], axis=1)

In [10]:
del df_concat['Summary']

In [11]:
df_concat.set_index('Player ID', drop=True, inplace=True)

In [12]:
df_concat = df_concat[['Player', 'Number', 'Flag', 'Position', 'Age', 'Height', 'Weight', 'Shoots/Catches',
                'Years Experience', 'Birth Date', 'Salary', 'Draft']]

In [13]:
df_concat['Flag'] = df_concat['Flag'].str.upper()
df_concat['Salary'] = df_concat['Salary'].apply(lambda x: x.replace('$', '')).apply(lambda x: x.replace(',', ''))
df_concat['Shoots/Catches'] = df_concat['Shoots/Catches'].apply(lambda x: 'L' if 'L' in x else 'R')

In [14]:
print(df_concat)

                        Player Number Flag Position Age Height Weight  \
Player ID                                                               
beaucfr01  Francois Beauchemin     23   CA        D  37   5-11    208   
berrare01           Reto Berra      1   CH        G  31    6-4    218   
biekske01         Kevin Bieksa      3   CA        D  36    6-1    197   
brownjt01           J.T. Brown     71   US       RW  27   5-10    169   
coglian01      Andrew Cogliano      7   CA       LW  30   5-10    177   
eavespa01        Patrick Eaves     18   CA       RW  33   5-10    202   
fowleca01           Cam Fowler      4   CA        D  26    6-1    215   
getzlry01     Ryan Getzlaf (C)     15   CA        C  32    6-4    223   
gibsojo02          John Gibson     36   US        G  24    6-3    223   
grantde01          Derek Grant     38   CA        C  27    6-3    215   
henriad01        Adam Henrique     14   CA        C  27    6-0    195   
kaseon01           Ondrej Kase     25   CZ       RW